In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
import plotly.express as px
pd.set_option('display.max_columns',None)

# Load data and remove duplicates

In [2]:
inv_mar = pd.read_csv('involved_markers_hebrew.csv',low_memory=False)
inv_mar['new_id'] = inv_mar['provider_and_id'].astype(str)+inv_mar['involve_id'].astype(str)
inv_mar.drop_duplicates('new_id',inplace=True)

## identify bus accidets involvers

In [3]:
bus_accidents = inv_mar[inv_mar['involve_vehicle_type_hebrew']=='אוטובוס']['provider_and_id'].unique()
inv_mar['bus'] = inv_mar['provider_and_id'].apply(lambda x:'אוטובוס' if(x in (bus_accidents)) else 'אחר')

### Separate the data to death pivot involvers and all the rest urban roads involvers

In [4]:
yishuv_name = ['פתח תקווה']
street_name = 'בוטינסקי'

d_pivot = inv_mar[(inv_mar['accident_yishuv_name'].isin(yishuv_name))&
                  ((inv_mar['street1_hebrew'].str.contains(street_name,na=False))|
                   (inv_mar['street2_hebrew'].str.contains(street_name,na=False)))].copy()

rest_urban = inv_mar[(~inv_mar['accident_yishuv_name'].isin(yishuv_name))&
                  ((~inv_mar['street1_hebrew'].str.contains(street_name,na=False))|
                   (~inv_mar['street2_hebrew'].str.contains(street_name,na=False)))&
                    (~inv_mar['road_type_hebrew'].str.contains('לא-עירונית'))].copy()

### Separate the data to fatalities and fatalities+severe injuries

In [5]:
d_piv_fatal = d_pivot[d_pivot['injury_severity_hebrew'].isin(['הרוג'])]
d_piv_severe = d_pivot[d_pivot['injury_severity_hebrew'].isin(['הרוג','פצוע קשה'])]

rest_fatal = rest_urban[rest_urban['injury_severity_hebrew'].isin(['הרוג'])]
rest_severe = rest_urban[rest_urban['injury_severity_hebrew'].isin(['הרוג','פצוע קשה'])]

# pedestrian Bus accidents

## fatal

In [6]:
year_filter = 2014
piv_df = d_piv_fatal
rest_df = rest_fatal

fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['ציר המוות', 'כל השאר-תאונות עירוניות'])
    
piv_gb = piv_df[(piv_df['accident_year']<=year_filter)&
              (piv_df['injured_type_hebrew']=='הולך רגל')]['bus'].value_counts()

rest_gb = rest_df[(rest_df['accident_year']<=year_filter)&
                (rest_df['injured_type_hebrew']=='הולך רגל')]['bus'].value_counts()

piv_colors = ['red','lightgray']
rest_colors = ['lightgray','red']

text = list(((rest_gb.values/sum(rest_gb.values))*100).round())
for i,x in enumerate(text):
    text[i] = str(int(x))+'%'
    
fig.add_trace(go.Pie(labels=piv_gb.index.values, values=piv_gb.values,marker=dict(colors=piv_colors)), 1, 1)
fig.add_trace(go.Pie(labels=rest_gb.index.values, values=rest_gb.values,text=text,textinfo='text',marker=dict(colors=rest_colors)), 1, 2)
fig.update_layout(title_text='?בין השנים 2008-{} איזה אחוז מהולכי הרגל שנהרגו נפגעו מאוטובוס'.format(year_filter))  
fig.show()

## severe+fatal

In [7]:
year_filter = 2014
piv_df = d_piv_severe
rest_df = rest_severe

fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['ציר המוות', 'כל השאר-תאונות עירוניות'])
    
piv_gb = piv_df[(piv_df['accident_year']<=year_filter)&
              (piv_df['injured_type_hebrew']=='הולך רגל')]['bus'].value_counts()

rest_gb = rest_df[(rest_df['accident_year']<=year_filter)&
                (rest_df['injured_type_hebrew']=='הולך רגל')]['bus'].value_counts()

colors = ['red','lightgray']
text = list(((rest_gb.values/sum(rest_gb.values))*100).round())
for i,x in enumerate(text):
    text[i] = str(int(x))+'%'
fig.add_trace(go.Pie(labels=piv_gb.index.values, values=piv_gb.values,marker=dict(colors=colors)), 1, 1)
fig.add_trace(go.Pie(labels=rest_gb.index.values, values=rest_gb.values,text=text,textinfo='text',marker=dict(colors=['lightgray','red',])), 1, 2)
fig.update_layout(title_text='?בין השנים 2008-{} איזה אחוז מהולכי הרגל שנפגעו קשה/נהרגו, נפגעו מאוטובוס'.format(year_filter))  
fig.show()

# Petah-Tikva Fatalities distribution

## identify death pivot accidets involvers

In [9]:
petah_tikva_involvers = inv_mar[inv_mar['accident_yishuv_name'] == 'פתח תקווה'].copy()

In [10]:
d_pivot_accidets = petah_tikva_involvers[(petah_tikva_involvers['street1_hebrew'].str.contains(street_name,na=False))|
                   (petah_tikva_involvers['street2_hebrew'].str.contains(street_name,na=False))]['provider_and_id'].unique()

petah_tikva_involvers['d_pivot'] = petah_tikva_involvers['provider_and_id'].apply(lambda x:'זבוטינסקי' if(x in (d_pivot_accidets)) else 'שאר הרחובות')

## filter fatalities

In [11]:
petah_tikva_fatal = petah_tikva_involvers[petah_tikva_involvers['injury_severity_hebrew'].isin(['הרוג'])]

## plotting the data
### Pie chart

In [12]:
year_filter = 2014

pt_gb = petah_tikva_fatal[petah_tikva_fatal['accident_year']<=year_filter].groupby('d_pivot')['d_pivot'].count()
pt_fig = go.Figure(go.Pie(labels=pt_gb.index.values, values=pt_gb.values))
pt_fig.update_traces(marker=dict(colors=colors))
pt_fig.update_layout(title_text='2008-{}  התפלגות ההרוגים בפ"ת לפי רחוב, בין השנים'.format(year_filter))  

### prepare the data for Area plot

In [13]:
pt_fatal_gb = petah_tikva_fatal.groupby(['d_pivot','accident_year'])['d_pivot'].count()

In [14]:
pt_fatal_by_year = petah_tikva_fatal.groupby('accident_year')['accident_year'].count()
d_piv_fatal_list = []
for year in range(2008,2019):
    try:
        d_piv_fatal_list.append(pt_fatal_gb['זבוטינסקי'][year])
    except:
        d_piv_fatal_list.append(0)

### Area plot

In [15]:
d_piv_trace = dict(type='scatter',
             x=list(range(2008,2019)),
             y=d_piv_fatal_list,
             mode='lines',
             name='הרוגים בזבוטינסקי',
             fill='tonexty',
             marker=dict(color='red'),      
             fillcolor = 'salmon')

rest_trace = dict(type='scatter',
             x=pt_fatal_by_year.index,
             y=pt_fatal_by_year.values,
             mode='lines',
             name='הרוגים בפ"ת',
             fill='tonexty',
             marker=dict(color='grey'),
             fillcolor = 'lightgrey')

close_trace = dict(type='scatter',
             x=[2014,2014],
             y=[0,13],
             mode='lines',
             name='סגירת ציר המוות',
             marker=dict(color='gold'),
             line=dict(width=4,dash='dash'))

traces = [d_piv_trace,rest_trace,close_trace] 
    
iplot(go.Figure(traces).update_layout(template='plotly_white',title='2008-2018  התפלגות ההרוגים בפ"ת לפי רחוב, בין השנים')) 

# Death Pivot fatalities and severe injuries plots

## Road Type

### fatal

In [18]:
year_filter = 2014
col = 'road_type_hebrew'
piv_df = d_piv_fatal
rest_df = rest_fatal

fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                subplot_titles=['ציר המוות', 'כל השאר-תאונות עירוניות'])
    
piv_gb = piv_df[piv_df['accident_year']<=year_filter][col].value_counts()
rest_gb = rest_df[rest_df['accident_year']<=year_filter][col].value_counts()

colors = ['red','lightgray']
fig.add_trace(go.Pie(labels=piv_gb.index.values, values=piv_gb.values), 1, 1)
fig.add_trace(go.Pie(labels=rest_gb.index.values, values=rest_gb.values), 1, 2)
fig.update_traces(marker=dict(colors=colors))
fig.update_layout(title_text='התפלגות ההרוגים לפי מיקום התאונה בין השנים 2008-{}'.format(year_filter))    
fig.show()

### fatal+severe

In [19]:
year_filter = 2014
col = 'road_type_hebrew'
piv_df = d_piv_severe
rest_df = rest_severe

fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                subplot_titles=['ציר המוות', 'כל השאר-תאונות עירוניות'])
    
piv_gb = piv_df[piv_df['accident_year']<=year_filter][col].value_counts()
rest_gb = rest_df[rest_df['accident_year']<=year_filter][col].value_counts()

colors = ['red','lightgray']
fig.add_trace(go.Pie(labels=piv_gb.index.values, values=piv_gb.values), 1, 1)
fig.add_trace(go.Pie(labels=rest_gb.index.values, values=rest_gb.values), 1, 2)
fig.update_traces(marker=dict(colors=colors))
fig.update_layout(title_text='התפלגות ההרוגים/פצועים קשה לפי מיקום התאונה בין השנים 2008-{}'.format(year_filter))    
fig.show()

### Traffic light condition
### fatal

In [20]:
year_filter = 2014
col = 'road_type_hebrew'
df = d_piv_fatal

fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                subplot_titles=['התפלגות ההרוגים', 'תקינות רמזור בצמתים'])
    
road_type = df[df['accident_year']<=year_filter]['road_type_hebrew'].value_counts()
traffic_light = df[df['accident_year']<=year_filter]['road_control_hebrew'].value_counts()

colors = ['red','lightgray']
fig.add_trace(go.Pie(labels=road_type.index.values, values=road_type.values,marker=dict(colors=colors)), 1, 1)
fig.add_trace(go.Pie(labels=traffic_light.index.values, values=traffic_light.values,hole=0.7,
                     marker=dict(colors=['lightgreen'])), 1, 2)
fig.update_layout(title_text='התפלגות ההרוגים בציר המוות לפי מיקום התאונה בין השנים 2008-{}'.format(year_filter))    
fig.show()

### fatal+severe

In [21]:
year_filter = 2014
col = 'road_type_hebrew'
df = d_piv_severe

fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                subplot_titles=['התפלגות ההרוגים', 'תקינות רמזור בצמתים'])
    
road_type = df[df['accident_year']<=year_filter]['road_type_hebrew'].value_counts()
traffic_light = df[df['accident_year']<=year_filter]['road_control_hebrew'].value_counts()

colors = ['red','lightgray']
fig.add_trace(go.Pie(labels=road_type.index.values, values=road_type.values,marker=dict(colors=colors)), 1, 1)
fig.add_trace(go.Pie(labels=traffic_light.index.values, values=traffic_light.values,hole=0.7,
                     marker=dict(colors=['lightgreen'])), 1, 2)
fig.update_layout(title_text='התפלגות ההרוגים/פצועים קשה בציר המוות לפי מיקום התאונה בין השנים 2008-{}'.format(year_filter))    
fig.show()

## Injured Type

### grouping injured type

In [24]:
def injured_type_group(x):
    categories = ['הולך רגל','רכב בעל 4 גלגלים ויותר','אופנוע','אופניים','רכב לא ידוע']
    for category in categories:
        if category in x:
            return category   
    return x    

In [25]:
dfs = [d_piv_fatal,rest_fatal,d_piv_severe,rest_severe]

for df in dfs:
    df['injured_type_group'] = df['injured_type_hebrew'].apply(injured_type_group)

C:\Users\Yuval\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### fatal

In [26]:
def get_colors(df):
    colors_list = ['red','lightgray','lightblue','Lavender ','LightSlateGray']
    colors = {}
    types_order = df.sum().sort_values(ascending=False).index
    for i,typ in enumerate(types_order):
        colors[typ] = colors_list[i]
    return colors    

In [27]:
df = d_piv_fatal
types_df = pd.DataFrame()
years = list(range(2008,2019))
types = df['injured_type_group'].unique()
for typ in types:
    values=[]
    for year in years:
        values.append(df[(df['injured_type_group']==typ)&(df['accident_year']==year)]['injured_type_group'].count())
    types_df[typ] = values

In [28]:
data = []
colors = get_colors(types_df)
layout = dict(title='התפלגות ההרוגים בציר המוות לפי סוג נפגע, בין השנים 2008-2018',
             xaxis={'title':'שנים'},
             yaxis={'title':'כמות הרוגים'},
             template='plotly_white')

types_order = types_df.sum().sort_values(ascending=False).index

for col in types_df:
    if col==types_order[0]:
        wid=5
    else:
        wid=3
        
    if col=='רכב לא ידוע':
        continue
    temp_trace = dict(type='scatter',
                  y=types_df[col],
                 x=years,
                 name=col,
                 mode='lines',
                 marker=dict(color=colors[col]),
                 line=dict(width=wid))
    data.append(temp_trace)  
    
close_trace = dict(type='scatter',
             x=[2014,2014],
             y=[0,4.5],
             mode='lines',
             name='סגירת ציר המוות',
             marker=dict(color='gold'),
             line=dict(width=4,dash='dash'))
data.append(close_trace)

iplot(go.Figure(data,layout).update_layout(xaxis = dict(
                                           tickmode = 'linear',
                                           tick0 = 2008,
                                           dtick = 1),
                                          yaxis = dict(
                                           tickmode = 'linear',
                                           tick0 = 1,
                                           dtick = 1)))     

In [29]:
data = []
colors = get_colors(types_df)
layout = dict(title='התפלגות ההרוגים בציר המוות לפי סוג נפגע, בין השנים 2008-2018',
             xaxis={'title':'שנים'},
             yaxis={'title':'כמות הרוגים'},
             template='plotly_white')
types_order = types_df.sum().sort_values(ascending=False).index
for col in types_order:
    if col=='רכב לא ידוע':
        continue    
    temp_trace = go.Bar(y=types_df[col],
                 x=years,
                 name=col,
                 marker=dict(color=colors[col]))
    data.append(temp_trace)  
    
close_trace = dict(type='scatter',
             x=[2014.5,2014.5],
             y=[0,5.5],
             mode='lines',
             name='סגירת ציר המוות',
             marker=dict(color='gold'),
             line=dict(width=4,dash='dash'))
data.append(close_trace)

iplot(go.Figure(data,layout).update_layout(dict(barmode = 'stack'),xaxis = dict(
                                           tickmode = 'linear',
                                           tick0 = 2008,
                                           dtick = 1),
                                          yaxis = dict(
                                           tickmode = 'linear',
                                           tick0 = 1,
                                           dtick = 1))) 

### fatal+severe

In [30]:
df = d_piv_severe
types_df = pd.DataFrame()
years = list(range(2008,2019))
types = df[df['injured_type_group']!='רכב לא ידוע']['injured_type_group'].unique()
types_df['type'] = types
types_df.set_index('type',inplace=True)

for year in years:
    values=[]
    for typ in types:
        values.append(df[(df['injured_type_group']==typ)&(df['accident_year']==year)]['injured_type_group'].count())
    types_df[str(year)] = values
    
for year in years:
    col_name = '{}_rank'.format(str(year))
    types_df[col_name] = types_df[str(year)].rank(method='min')  

In [32]:
df = d_piv_severe
types_df = pd.DataFrame()
years = list(range(2008,2019))
types = df['injured_type_group'].unique()
for typ in types:
    values=[]
    for year in years:
        values.append(df[(df['injured_type_group']==typ)&(df['accident_year']==year)]['injured_type_group'].count())
    types_df[typ] = values

In [33]:
data = []
colors = get_colors(types_df)
layout = dict(title='התפלגות ההרוגים/פצועים קשה בציר המוות לפי סוג נפגע, בין השנים 2008-2018',
             xaxis={'title':'שנים'},
             yaxis={'title':'כמות הרוגים/פצועים קשה'},
             template='plotly_white')

types_order = types_df.sum().sort_values(ascending=False).index

for col in types_df:
    if col==types_order[0]:
        wid=5
    else:
        wid=3
        
    if col=='רכב לא ידוע':
        continue
    temp_trace = dict(type='scatter',
                  y=types_df[col],
                 x=years,
                 name=col,
                 mode='lines',
                 marker=dict(color=colors[col]),
                 line=dict(width=wid))
    data.append(temp_trace)  
    
close_trace = dict(type='scatter',
             x=[2014,2014],
             y=[0,6.5],
             mode='lines',
             name='סגירת ציר המוות',
             marker=dict(color='gold'),
             line=dict(width=4,dash='dash'))
data.append(close_trace)

iplot(go.Figure(data,layout).update_layout(xaxis = dict(
                                           tickmode = 'linear',
                                           tick0 = 2008,
                                           dtick = 1),
                                          yaxis = dict(
                                           tickmode = 'linear',
                                           tick0 = 1,
                                           dtick = 1)))  

In [44]:
data = []
colors = get_colors(types_df)
layout = dict(title='התפלגות ההרוגים/פצועים קשה בציר המוות לפי סוג נפגע, בין השנים 2008-2018',
             xaxis={'title':'שנים'},
             yaxis={'title':'כמות הרוגים/פצועים קשה'},
             template='plotly_white')

types_order = types_df.sum().sort_values(ascending=False).index
for col in types_order:
    temp_trace = go.Bar(y=types_df[col],
                 x=years,
                 name=col,
                 marker=dict(color=colors[col]))
    data.append(temp_trace)  
    
close_trace = dict(type='scatter',
             x=[2014.5,2014.5],
             y=[0,9.3],
             mode='lines',
             name='סגירת ציר המוות',
             marker=dict(color='gold'),
             line=dict(width=4,dash='dash'))
data.append(close_trace)

iplot(go.Figure(data,layout).update_layout(dict(barmode = 'stack'),xaxis = dict(
                                           tickmode = 'linear',
                                           tick0 = 2008,
                                           dtick = 1),
                                          yaxis = dict(
                                           tickmode = 'linear',
                                           tick0 = 1,
                                           dtick = 1))) 